In [1]:
import os, sys
import numpy as np

from eggd800.signal import demux, butter_lowpass_filter  # for demuxing and filtering
import scipy.io.wavfile  # for reading in audiofiles

import parselmouth as ps

This notebook assumes that the raw WAV files are kept in a subfolder named 'raw', and will output calibrated files in a subfolder called 'calibrated'.

In [2]:
fromdir = os.path.abspath('./raw')
todir = os.path.abspath('./calibrated')

In [3]:
files = [f for f in os.listdir(fromdir) if f[-4:]=='.wav']  # get a list of the raw audio files
files

['luciana_20190616.wav', 'marquinho_20190616.wav', 'jako_20190616.wav']

Set values for smoothing and calibration

In [4]:
# These are default smoothing values
cutoff = 50
order = 3

In [5]:
# This is a dictionary containing spans of time in the audiofile where the mask is not on, in order
# to determine the best intercepts/zeroes for each recording. Where the span is greater than 50ms, it 
# has been arbitrarily limited to 50ms (to reduce processing time)
zerospans = {
    'luciana': [150,200],
    'jako': [250, 300],
    'marquinho': [118, 140]
}

In [6]:
# These are mask sizes of the subjects
mask = {
    'luciana': 'small',
    'jako': 'small',
    'marquinho': 'small'
}

# These are the previously determined calibration slopes and intercepts from previous Panara calibration
big_calib ={
    'nas_slope': 3900,
    'nas_interc': 274,
    'ora_slope': 323,
    'ora_interc':763
}

small_calib ={
    'nas_slope': 7456,
    'nas_interc': 206,
    'ora_slope': 801,
    'ora_interc': 755
}

This next cell loops through each wav file in the raw data folder and does the following:
1. separates the 2 channels into 4, performs smoothing on each channel, and then recombines them into a 3-channel wav file, with audio, nasal airflow, and oral airflow, saved as tmp.wav in the current directory
1. reads tmp.wav as a Parselmouth Sound object
1. determines the average value during the zerospans -- periods of time when the recording should read zero for both oral and nasal airflow
1. calibrates the channels based on the slopes from previous calibration and intercepts determined in the previous step
1. recombines the channels into a wav file with the original name plus the suffix `_calibrated` in the target directory

In [7]:
for f in files:
    
    destfile = os.path.join(todir,f[0:-4]+'_calibrated'+'.wav') # name the destination file
    
    (au, lx, p1, p2) = demux(scipy.io.wavfile.read(os.path.join(fromdir,f))[1]) # separate channels
    subject = f.split('_')[0] # get subject -- needed for calibration

    # get the sampling frequency of the audiofile, to reconstruct time
    # this has to be halved because there actually four channels, not two
    rate = scipy.io.wavfile.read(os.path.join(fromdir,f))[0]/2

    af_n = butter_lowpass_filter(p1, cutoff, rate, order)  # nasal airflow on p1
    af_o = butter_lowpass_filter(p2, cutoff, rate, order)  # oral airflow on p2

    # recombine channels and write to a temporary wav file
    ch_au = np.asarray(au/max(abs(au)), dtype=np.float32)
    ch_n = np.asarray(af_n, dtype=np.float32)
    ch_o = np.asarray(af_o, dtype=np.float32)
    chs = np.vstack((ch_au, ch_n, ch_o)).T
    scipy.io.wavfile.write('./tmp.wav', rate=int(rate), data=chs)

    # read tmp.wav and get the average values of nasal and oral airflow channels when it should be zero
    sound = ps.read('./tmp.wav')
    zerosound = sound.extract_part(zerospans[subject][0], zerospans[subject][1])
    nas_interc = np.mean(zerosound.as_array()[1])
    ora_interc = np.mean(zerosound.as_array()[2])
    print('\t'.join([subject,str(nas_interc),str(ora_interc)])) # print the zeroes, for fun
    
    # calibrate!
    if mask[subject] == 'small':
        af_o = (af_o-ora_interc)/small_calib['ora_slope']
        af_n = (af_n-nas_interc)/small_calib['nas_slope']
    else:
        af_o = (af_o-ora_interc)/big_calib['ora_slope']
        af_n = (af_n-nas_interc)/big_calib['nas_slope']

    # recombine channels and write to final calibrated file
    ch_n = np.asarray(af_n, dtype=np.float32)
    ch_o = np.asarray(af_o, dtype=np.float32)
    chs = np.vstack((ch_au, ch_n, ch_o)).T
    scipy.io.wavfile.write(destfile, rate=int(rate), data=chs)

luciana	457.42245590487164	761.023029610494
marquinho	453.7605006361297	761.1289517435016
jako	493.26027835906984	764.5326677579752
